## File : Exercise 12-2_Edris_Safari.ipynb
## Name:Edris Safari
## Date:2/09/2019
## Course: DSC530 - Data Exploration and Analysis
## Desc: Week8 exercise 12-2 assignment


Write a definition for a class named `SerialCorrelationTest` that extends `HypothesisTest` from Section 9.2. It should take a series and a lag as data, compute the serial correlation of the series with the given lag, and then compute the p-value of the observed correlation.

Use this class to test whether the serial correlation in raw price data is statistically significant. Also test the residuals of the linear model and (if you did the previous exercise), the quadratic model.

In [22]:
from __future__ import print_function, division

%matplotlib inline

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np
import pandas as pd

import statsmodels.formula.api as smf

import random

import thinkstats2
import thinkplot
import timeseries


## Serial correlation

The following function computes serial correlation with the given lag.

In [19]:
def SerialCorr(series, lag=1):
    xs = series[lag:]
    ys = series.shift(lag)[lag:]
    corr = thinkstats2.Corr(xs, ys)
    return corr

In [20]:
class SerialCorrelationTest(thinkstats2.HypothesisTest):
    """Tests serial correlations by permutation."""

    def TestStatistic(self, data):
        """Computes the test statistic.
        The test statistic runs correlation between each value and it's lagged value

        data: tuple of xs and ys
        """
        series, lag = data
        test_stat = abs(SerialCorr(series, lag))
        return test_stat

    def RunModel(self):
        """Run the model of the null hypothesis.
        It is run 1000 times to compute the p-value

        returns: simulated data
        """
        series, lag = self.data
        permutation = series.reindex(np.random.permutation(series.index))
        return permutation, lag
    

In [23]:
def RunQuadraticModel(daily):
    """Runs a Quadratic model of price per gram versus years and years squared.

    daily: DataFrame of daily prices

    returns: model, results
    """
    # Create a new column for the model
    daily['years2'] = daily.years**2
    model = smf.ols('ppg ~ years + years2', data=daily)
    results = model.fit()
    return model, results

## Time series analysis

Load the data from "Price of Weed".

In [24]:
transactions = pd.read_csv('mj-clean.csv', parse_dates=[5])
transactions.head()

,city,state,price,amount,quality,date,ppg,state.name,lat,lon
0,Annandale,VA,100,7.075,high,2010-09-02,14.13,Virginia,38.830345,-77.213870
1,Auburn,AL,60,28.300,high,2010-09-02,2.12,Alabama,32.578185,-85.472820
2,Austin,TX,60,28.300,medium,2010-09-02,2.12,Texas,30.326374,-97.771258
3,Belleville,IL,400,28.300,high,2010-09-02,14.13,Illinois,38.532311,-89.983521
4,Boone,NC,55,3.540,high,2010-09-02,15.54,North Carolina,36.217052,-81.687983


`dailies` is the map from quality name to DataFrame.

In [53]:
dailies = timeseries.GroupByQualityAndDay(transactions)

In [54]:


# test the correlation between consecutive prices

name = 'high'
daily = dailies[name]

series = daily.ppg
# For daily price of high quality weed and lag of 1, compute the serial correlation
test = SerialCorrelationTest((series, 1))

The actual value of correlation is computed by SerialCorrelationTest's  TestStatistic and the p-value is computed by running TestStatistic 1000 times with RunModel producing a permuted series every time.

In [43]:
pvalue = test.PValue()
print(test.actual, pvalue)

0.48522937619473827 0.0


p-value is zero which shows no serial correlation. The actual correlation is .48 

In [47]:
test = SerialCorrelationTest((series, 20))
pvalue = test.PValue()
print(test.actual, pvalue)

0.48669484708192995 0.0


In [48]:
test = SerialCorrelationTest((series, 200))
pvalue = test.PValue()
print(test.actual, pvalue)

0.3984215588187465 0.0


In [49]:
test = SerialCorrelationTest((series, 365))
pvalue = test.PValue()
print(test.actual, pvalue)

0.3403450179460918 0.0


The actual correaltion decreases with lag value.

In [52]:
# test for serial correlation in residuals of the linear model

_, results = timeseries.RunLinearModel(daily)
series = results.resid
test = SerialCorrelationTest((series, 1))
pvalue = test.PValue()
print(test.actual, pvalue)    

0.07570473767506267 0.009


p-value increased a little bit but not significantly

In [51]:
# test for serial correlation in residuals of the quadratic model

_, results = RunQuadraticModel(daily)
series = results.resid
test = SerialCorrelationTest((series, 1))
pvalue = test.PValue()
print(test.actual, pvalue)

0.056073081612899055 0.044


p-value increased a little bit but not significantly. 